In [1]:
from pandas import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [2]:
df = pd.read_csv('NationalNames.csv')
df = df.drop_duplicates(subset="Name")
df.head()

Id       Name  Year Gender  Count
0   1       Mary  1880      F   7065
1   2       Anna  1880      F   2604
2   3       Emma  1880      F   2003
3   4  Elizabeth  1880      F   1939
4   5     Minnie  1880      F   1746

In [3]:
# Check if the name ends in vowel
def checkVowelEnd(name):
    if name[-1] in "aeiou":
        return "Vowel End"
    return "Consonant End"

In [4]:
df["Vowel/Consonant End"] = df["Name"].apply(checkVowelEnd)
df.head()

Id       Name  Year Gender  Count Vowel/Consonant End
0   1       Mary  1880      F   7065       Consonant End
1   2       Anna  1880      F   2604           Vowel End
2   3       Emma  1880      F   2003           Vowel End
3   4  Elizabeth  1880      F   1939       Consonant End
4   5     Minnie  1880      F   1746           Vowel End

In [5]:
def checkGender(gender):
    if gender == "F":
        return 0
    else:
        return 1
    
df["Gender Value"] = df["Gender"].apply(checkGender)

df.head(10)

Id       Name  Year Gender  Count Vowel/Consonant End  Gender Value
0   1       Mary  1880      F   7065       Consonant End             0
1   2       Anna  1880      F   2604           Vowel End             0
2   3       Emma  1880      F   2003           Vowel End             0
3   4  Elizabeth  1880      F   1939       Consonant End             0
4   5     Minnie  1880      F   1746           Vowel End             0
5   6   Margaret  1880      F   1578       Consonant End             0
6   7        Ida  1880      F   1472           Vowel End             0
7   8      Alice  1880      F   1414           Vowel End             0
8   9     Bertha  1880      F   1320           Vowel End             0
9  10      Sarah  1880      F   1288       Consonant End             0

In [6]:
def compare(group):
    return df.groupby([group])["Gender Value"].sum()*100/df.groupby([group])["Gender Value"].count()

In [7]:
df.groupby(["Vowel/Consonant End"])['Gender Value'].count()

Vowel/Consonant End
Consonant End    43635
Vowel End        50254
Name: Gender Value, dtype: int64

In [8]:
print (len(df))
# > 93889 = 43635 + 50254

93889


In [9]:
female_names = sum(df.groupby(["Vowel/Consonant End"])["Gender Value"].sum())
all_names = df.groupby(["Gender"])["Gender Value"].count()
print (all_names)
print ("\nBoth are equal? %s" % str(female_names == all_names["F"]))

Gender
F    59930
M    33959
Name: Gender Value, dtype: int64

Both are equal? False


In [10]:
print(df.groupby(["Vowel/Consonant End"])["Gender Value"].sum()*100/df.groupby(["Vowel/Consonant End"])["Gender Value"].count())

Vowel/Consonant End
Consonant End    59.880830
Vowel End        15.580849
Name: Gender Value, dtype: float64


In [11]:
print(compare("Vowel/Consonant End"))

Vowel/Consonant End
Consonant End    59.880830
Vowel End        15.580849
Name: Gender Value, dtype: float64


In [12]:
def vowelConsonantStart(name):
    if name[0] in "aeiou":
        return "Vowel Start"
    else:
        return "Consonant Start"

df["Vowel/Consonant Start"] = df["Name"].apply(vowelConsonantStart)

print("\n Comparison => %s", compare("Vowel/Consonant Start"))

df.head()


 Comparison => %s Vowel/Consonant Start
Consonant Start    36.169306
Name: Gender Value, dtype: float64


Id       Name  Year Gender  Count Vowel/Consonant End  Gender Value  \
0   1       Mary  1880      F   7065       Consonant End             0   
1   2       Anna  1880      F   2604           Vowel End             0   
2   3       Emma  1880      F   2003           Vowel End             0   
3   4  Elizabeth  1880      F   1939       Consonant End             0   
4   5     Minnie  1880      F   1746           Vowel End             0   

  Vowel/Consonant Start  
0       Consonant Start  
1       Consonant Start  
2       Consonant Start  
3       Consonant Start  
4       Consonant Start

In [13]:
def shortLongName(name):
    if len(name) < 7:
        return "Short"
    else:
        return "Long"

df["Short/Long Name"] = df["Name"].apply(shortLongName)
print(compare("Short/Long Name"))
df.head()

Short/Long Name
Long     29.360484
Short    42.645470
Name: Gender Value, dtype: float64


Id       Name  Year Gender  Count Vowel/Consonant End  Gender Value  \
0   1       Mary  1880      F   7065       Consonant End             0   
1   2       Anna  1880      F   2604           Vowel End             0   
2   3       Emma  1880      F   2003           Vowel End             0   
3   4  Elizabeth  1880      F   1939       Consonant End             0   
4   5     Minnie  1880      F   1746           Vowel End             0   

  Vowel/Consonant Start Short/Long Name  
0       Consonant Start           Short  
1       Consonant Start           Short  
2       Consonant Start           Short  
3       Consonant Start            Long  
4       Consonant Start           Short

In [14]:
training_data = df[["Gender Value", "Vowel/Consonant Start", "Short/Long Name", "Vowel/Consonant End"]]
training_data.head()

Gender Value Vowel/Consonant Start Short/Long Name Vowel/Consonant End
0             0       Consonant Start           Short       Consonant End
1             0       Consonant Start           Short           Vowel End
2             0       Consonant Start           Short           Vowel End
3             0       Consonant Start            Long       Consonant End
4             0       Consonant Start           Short           Vowel End

In [16]:
def reprCategory(column):
    column = column.astype("category")
    return column.cat.codes
    
training_data[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]] = training_data[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]].apply(reprCategory)

training_data.head()

C:\Users\IMF\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Gender Value  Vowel/Consonant Start  Short/Long Name  Vowel/Consonant End
0             0                      0                1                    0
1             0                      0                1                    1
2             0                      0                1                    1
3             0                      0                0                    0
4             0                      0                1                    1

In [17]:
train, test = train_test_split(training_data, test_size = 0.20)

In [18]:
clf = DecisionTreeClassifier()

In [19]:
clf = clf.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"])

In [20]:
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [21]:
clf.feature_importances_

array([0.95045077, 0.04954923, 0.        ])

In [22]:
predictions = clf.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
accuracy_score(test["Gender Value"], predictions)

0.7316008094578762

In [23]:
with open("decidenames.dot", "w") as dot_file:
    dot_file = export_graphviz(clf,
                            feature_names=["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"], out_file=dot_file)